In [1]:
from comet_ml import Experiment

from log_utils import plot_mean_std_loss, GroupedCometLogger
from data_reader import DataReader
from autoencode import Autoencoder

from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

from sklearn.preprocessing import StandardScaler

import numpy as np
from sklearn.model_selection import GroupKFold, GroupShuffleSplit

Using TensorFlow backend.


In [2]:
filenames = ["X1_train.csv", "X2_train.csv", "X3_train.csv"]
data_reader = DataReader(data_set_filenames=filenames, groups_filename="ID_train.csv")
data = data_reader.get_all_data().values

groups = data_reader.get_groups()

train_idx, val_idx = next(GroupShuffleSplit().split(data, groups=groups))
data_train, data_val = data[train_idx], data[val_idx]

In [3]:
sc = StandardScaler()
data_train = sc.fit_transform(data_train)
data_val = sc.transform(data_val)

In [25]:
data_train.shape

(160, 3048)

In [26]:
pca = PCA(n_components=3048)
pca.fit(data_train)


embedding = pca.transform(data_train)
predictions = pca.inverse_transform(embedding)


val_rmse = np.sqrt(((predictions - data_train)**2).mean())
print(val_rmse)

2.0196012525810603e-15


COMET ERROR: Streaming error


In [ ]:
input_shape = (data.shape[1],)
latent_shape = (100,)

config = {
    "encoder": [
        {
            "name": "latent",
            "type": "Dense",
            "kwargs": {
                "units": latent_shape[0],
                "activation": "linear"
            }
        }
    ],
    "decoder": [
        {
            "name": "output",
            "type": "Dense",
            "kwargs": {
                "units": data.shape[1],
                "activation": "linear"
            }
        }
    ]
}

optimizer_params = {
    "type": "adam",
    "kwargs": {
        
    }
}

ae = Autoencoder(config["encoder"],
                 config["decoder"],
                 input_shape=input_shape,
                 latent_shape=latent_shape,
                 loss="mean_squared_error",
                 optimizer_params=optimizer_params)

groups = data_reader.get_groups()

experiment = Experiment(project_name="comet test", api_key="50kNmWUHJrWHz3FlgtpITIsB1")
experiment.log_parameter("Experiment name", "PCA comparison test")

ae.fit(data_train, epochs=10000)
predictions = ae.predict(data_val)

val_rmse = np.sqrt(((predictions - data_val)**2).mean())
print(val_rmse)

### 